# Morteza's Test problems

## Question 1: Ziggurat Method

### Learning objectives
In this question you will:

- understand and implement a commonly used algorithm to sample probability distributions
- analyse the performance of this algorithm and compare with other algorithms


The Marsaglia-Tsang _ziggurat method_ is a intended as a fast technique for generating single-precision deviates from a continuous, monotonically decreasing PDF on $x \ge 0$.  (If the actual PDF is a two-sided but even function of $x$, and decreasing in $|x|$—a very common situation, as in Cauchy, Laplace, or normal distributions, for example—then we can use the ziggurat method to produce deviates for $x \ge 0$ assuming a one-sided distribution, then randomly choose the overall sign with $50\%/50\%$ probability).  It is basically a table-based rejection method turned on its side, which can typically achieve very low rejection probability, and with additional savings due to a "squeeze" on the acceptance boundary.  As usual, speed is achieved at the expense of additional storage and one-time initialization.  But savings can be significant compared to a transformation method if the needed inverse function is slow.

Let us illustrate this technique by developing a routine for sampling exponential deviates, which we can compare to the logarithmic transformation method.
<div style="width: 500px;margin: auto" align="center">
    <br>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/The_10-story_Ziggurat%2C_a_pyramidal_state_office_building_along_the_Sacramento_River%2C_across_from_downtown_Sacramento%2C_California%27s_capital_city_LCCN2013630972.tif/lossy-page1-1280px-thumbnail.tif.jpg">
    Ziggurats were popular in ancient Mesopotamian architecture. Pictured above is <a href="https://en.wikipedia.org/wiki/The_Ziggurat">a modern building</a> in Sacramento built in the same style.
</div>

Suppose the target deviates are distributed according to the pdf $f(x) = e^{-x}$ for all $x \ge 0$.  Consider a plot of $f(x)$ in the right half of the $(x,y)$ plane.  We divide the full vertical range of $f(x)$, namely $\lim\limits_{x \to \infty} f(x) = 0 \le y \le 1 =  f(0)$, into $N$ rectangular strips, plus one residual non-rectangular region near the $x$-axis.  These regions are defined by:

\begin{align}
0  \le x &\le x_{N-1}, \,   		&& f(x_{N-1}) = y_{N-1} \le y <  y_N = f(x_N) = 1, \\
0  \le x &\le x_{N-2}, \,   		&& f(x_{N-2}) = y_{N-2} \le y < y_{N-1} = f(x_{N-1}), \\
&\dotsb 	\\
0  \le x &\le x_{1}, \,  			&& f(x_{1}) = y_{1} \le y <  y_{2} = f(x_{2}) , \\
0 \le x &< \infty,\,         && 0 \le y < \min[ y_1, e^{-x} ] ,
\end{align}

along with the conditions that the _areas_ inside all $N$ regions should be equal (up to some acceptably small numerical error):
$$
A_0 = A_1 = \dotsb = A_{N-2} = A_{N-1}=A,
$$
where

\begin{align}
A_{N-1} &= x_{N-1} (y_{N} - y_{N-1}),\\
A_{N-2} &= x_{N-2} (y_{N-1} - y_{N-2}) , \\
&\dotsb \\
A_{1} &= x_{1} (y_{2}-y_{1}) ,\\
A_{0} &= x_{1} \, y_{1} \,+\, \int_{x_{1}}^{\infty} \!  e^{-x} \, dx =  x_{1} \, y_{1} + e^{-x_{1}}.
\end{align}

The union of these strips is the "ziggurat" lending the method its name.  The basic idea is to sample a point $(x,y)$ _uniformly_ within the ziggurat, but reject and repeat if it also lies to the right of the $f(x)$ curve.  The $x$ coordinate of an accepted point will then be sampled from $f(x)$.
<br>
<div style="width: 500px;margin: auto" align="center">
    <img src="ziggurat.png">
    Illustration of the ziggurat method, for $N=10$. The black line is the target distribution, the red equal-area strips together form the ziggurat, which essentially constitute the prior distiribution for rejection sampling. The green region is the area where no computation is required, which provides a speed-up. The method is this: pick a random strip, and pick a random $x$ on that strip. If it's below the next strip (i.e. in the green region), return it, otherwise use a rejection or transformation method.
</div>

Since the strips all have equal area, the first step is to use a random integer $i \in \{0, \dotsc, N-1\}$ to choose a strip at random, with probability $p_i = \tfrac{1}{N}$. (Ideally we would use a power of 2, e.g. $N=256$, so that we may simply choose an 8-bit integer.)

Suppose we find ourselves in the $i$th strip.  We are then to choose a point uniformly at random (with respect to area) within the chosen strip.  Notice that if the point were to fall anywhere directly below the strip directly above, then it must lie to the left of $f(x)$ within the current strip, and hence should be accepted regardless of the particular value of $y$.  This "squeeze" that can be imposed on the accept/reject boundary greatly enhances the efficiency of the method, because often the trial point can be accepted without sampling of a second deviate or calculation of any logs or exponentials.

So we next choose a uniform random variate $k \in [0,1)$ and compute $x = kx_i$. If $x<x_{i+1}$, we can output $x$ without ever needing to determine $y$. For $i=0$ we define $x_0=\frac{A}{y_1}$ to map the rectangular part of the strip to the interval $[0,\frac{x_1y_1}{A})$ in $k$.

Otherwise, we generate another uniform random variate $u \in [0,1)$ (or recycle the remaining bits of precision in $k$).
If $i = 0$, then by now we know that $x$ should fall under the exponential tail in this bottom strip, so we output $x = x_1 -\ln u$.  If instead $i > 0$, we accept the sample and output $x  =  k x_i$ if $u < \tfrac{ e^{-x} - y_i}{y_{i + 1} - y_i}$, but otherwise reject $x$ and start over with a new choice of $i$ and $k$.

### 1a. 

Estimate the expected number of exponential or logarithm functions that need to be evaluated per output variate, in terms of $N$ and $A$.

Write your answer here

### 1b. 

It remains to figure out the values of $x_i$, which are to be pre-computed and stored.  This can be done by an iterative bisection search.  Starting with a trial value of $x_1$, calculate $A$, and then march upward, carving off successive rectangular strips of area $A$, and whose bottom right corners lie on $f(x)$.  If $x_1$ is chosen too large, and hence $A$ is too small, then we end up with $y_{N} < 1$.  If instead $x_1$ is chosen too small, and hence $A$ is too large, then we reach a $y_i \ge 1$ for some $i < N$.  We can then bracket the correct value of $x_1$ between a pair of values (one too large and one too small), and then refine this interval by bisection until the error falls below some small tolerance.

NOTE:  this is only fast if the tables can fit into cache; otherwise, even re-calculation of transcendental functions may be faster than slow memory look-ups.

Fill in the following function that does this for the exponential distribution, given $N$. 

In [ ]:
#Write your answer here

### 1c. 

Using the function you filled in the previous part, and your estimate in the first part, make a plot of the expected number of function computations per output variate as a function of $N$.

In [ ]:
#Write your answer here

### 1d. 

Fill in the methods below to generate samples from the exponential distribution using the ziggurat method. For $N=10$, draw a large number of samples and compare the expected distribution to a histogram.

In [ ]:
#Write your answer here

### 1e. 

Using a large number of samples, compare the speed of this algorithm to the logarithmic transformation method that is the conventional choice for generating exponential deviates. 

In [ ]:
#Write your answer here

---

## Question 2: Track Reconstruction and Vertex Fitting

### Learning objectives
In this question you will:

- Introduce the concept of track fitting using the example of a straight line fit
- Understand the meaning of track impact parameter and calculate the dependence of impact parameter resolution on the relevant detector parameters
- Use Monte Carlo techniques to demonstrate that the decay products of long-lived particles can be "tagged" by selecting tracks with high impact parameter
- Use the example of a simple two-body decay to introduce the topic of secondary vertex reconstruction


Silicon microstrip and pixel detectors play a prominent role in many particle physics experiments.  Their good spatial resolution, ability to distinguish nearby particles and radiation hardness make them attractive options.  You have studied the position resolution of such detectors in a previous homework.  Today, we will learn about how they are used to reconstruct particle trajectories and to identify the decay products of weakly decaying particles.

Track reconstruction is performed in several steps.  First, neighboring hits are combined to form space-points.  Second, a pattern recognition algorithm is used to associate a set of space-points with a single trajectory.  Finally, these space-points are fit to determine track parameters.  This problem focuses on the final stage where track parameters are determined.

In this problem we will consider the case of a charged particle traversing multiple silicon strip detectors leaving a straight line trajectory.  An example of this case is the [LHCb VErtex LOcator (VELO) detector](https://lhcb-public.web.cern.ch/lhcb-public/en/detector/VELO2-en.html).  In addition, while the ATLAS and CMS trackers are placed in a solenoidal magnetic field, fits in the non-bending plane ($r$-$z$ in cylindrical coordinates with the $\vec{B}$ field along the $z$-axis) will be straight lines.



Consider a very simplified model of a tracking detector:

<div style="width: 500px;margin: auto" align="center">
    <img src="tracker.png">
    Simplified model of a tracking detector.
</div>

The detector will consist of 4 layers of silicon equally spaced with the first layer a distance $z_0$ from the beam interaction point and a $z$-spacing of $l$ between each layer.  The $z$ position of the detectors is perfectly known.  The strips are oriented to give a measurement of $r$ in each detector with a resolution $\sigma_r$.  The track is a straight line, so it can be described as a trajectory in the $r-z$ plane

$$r = az + b$$

where the best estimates of $a$ and $b$ are obtained using a $\chi^2$ fit to the 4 spacepoints.


### 2a. 

If $r_0$ through $r_3$ are the measured hit positions on the 4 layers of silicon, find analytic expressions for the fit values of $a$ and $b$.  Express your answer in terms of the $z$ positions of the sensors and the measured $r$ positions.

Write your answer here

### 2b. 

Find an expression for the uncertainty on the intercept $b$.

Write your answer here

### 2c. 

Generate 1000 tracks all coming from the beam interaction point (these are called "primary tracks") and distributed uniformly in $\cos \theta$ for $-0.25 < \theta < 0.25$.  For each track, simulate the measured hit positions using a resolution $\sigma_r = 15$ $\mu$m and fit the measured hits to determine $m$ and $b$.  Use the values $z_0 = 5$ cm and $l = 10$ cm.  Make a histogram of the fit values of $b$ for the tracks and verify that the width of the distrbiution agrees with your analytic result above.

In [ ]:
#Write your answer here